In [1]:
import numpy as np
import pandas as pd
import torch
_ = torch.manual_seed(10)
import os
import math
from sbi import utils as utils
import sbi
from sbi import inference
from sbi.inference import SNPE, prepare_for_sbi, simulate_for_sbi
import scipy.io as sio

device = torch.device("cpu")

In [2]:
class CustomPriorDist:
    def __init__(self,a,b, return_numpy: bool = False):
        self.dist1 = utils.BoxUniform(a,b)
        self.dist2 = utils.BoxUniform(a,b)
        self.dist3 = utils.BoxUniform(a,50*b)
        self.dist4 = utils.BoxUniform(a,500*b)
        self.dist5 = utils.BoxUniform(b,32*b)
        self.dist6 = utils.BoxUniform(a,b)
        self.dist7 = utils.BoxUniform(a,b)
        self.dist8 = utils.BoxUniform(a,50*b)
        self.dist9 = utils.BoxUniform(a,500*b)
        self.return_numpy = return_numpy
        
    def sample(self, sample_shape=torch.Size([])):
        if len(sample_shape) == 1:
            length = sample_shape[0]
            samples = torch.ones(length,9)
            temp_1 = self.dist1.sample(sample_shape)
            temp_2 = self.dist2.sample(sample_shape)
            temp_3 = self.dist3.sample(sample_shape)
            temp_4 = self.dist4.sample(sample_shape)
            temp_5 = self.dist5.sample(sample_shape)
            temp_6 = self.dist6.sample(sample_shape)
            temp_7 = self.dist7.sample(sample_shape)
            temp_8 = self.dist8.sample(sample_shape)
            temp_9 = self.dist9.sample(sample_shape)
            samples[:,0] = temp_1[:,0]
            samples[:,1] = temp_2[:,0]
            samples[:,2] = temp_3[:,0]
            samples[:,3] = temp_4[:,0]
            samples[:,4] = temp_5[:,0]
            samples[:,5] = temp_6[:,0]
            samples[:,6] = temp_7[:,0]
            samples[:,7] = temp_8[:,0]
            samples[:,8] = temp_9[:,0]
            return samples.numpy() if self.return_numpy else samples
        else:
            samples = torch.ones(1,9)
            temp_1 = self.dist1.sample(sample_shape)
            temp_2 = self.dist2.sample(sample_shape)
            temp_3 = self.dist3.sample(sample_shape)
            temp_4 = self.dist4.sample(sample_shape)
            temp_5 = self.dist5.sample(sample_shape)
            temp_6 = self.dist6.sample(sample_shape)
            temp_7 = self.dist7.sample(sample_shape)
            temp_8 = self.dist8.sample(sample_shape)
            temp_9 = self.dist9.sample(sample_shape)

            samples[:,0] = temp_1[0]
            samples[:,1] = temp_2[0]
            samples[:,2] = temp_3[0]
            samples[:,3] = temp_4[0]
            samples[:,4] = temp_5[0]
            samples[:,5] = temp_6[0]
            samples[:,6] = temp_7[0]
            samples[:,7] = temp_8[0]
            samples[:,8] = temp_9[0]
            return samples.numpy() if self.return_numpy else samples            
    
    def log_prob(self, values):
        log_probs = torch.ones((values.size()[0],))
        length = values.size()[0]
        if self.return_numpy:
            values = torch.as_tensor(values)
        
        for i in range(values.size()[0]):
            temp = torch.ones(9)
            temp[0] = self.dist1.log_prob(values[i][0])
            temp[1] = self.dist2.log_prob(values[i][1])
            temp[2] = self.dist3.log_prob(values[i][2])
            temp[3] = self.dist4.log_prob(values[i][3])
            temp[4] = self.dist5.log_prob(values[i][4])
            temp[5] = self.dist6.log_prob(values[i][5])
            temp[6] = self.dist7.log_prob(values[i][6])
            temp[7] = self.dist8.log_prob(values[i][7])
            temp[8] = self.dist9.log_prob(values[i][8])
            log_probs[i] =  torch.sum(temp)
            
        return log_probs.numpy() if self.return_numpy else log_probs

In [3]:
initial_data = sio.loadmat("Neural_method_initial.mat")

In [4]:
theta_np = initial_data["part_vals_prior"]
sims_np = initial_data["prior_pred_sim"]
obs_np = initial_data["syn4"]

In [5]:
theta = torch.from_numpy(theta_np).to(torch.float32).to(device)
x = torch.from_numpy(sims_np).to(torch.float32).to(device)
x_0 = torch.from_numpy(obs_np).to(torch.float32).to(device)

In [6]:
priorDist = CustomPriorDist(torch.zeros(1).to(device),torch.ones(1).to(device))
def simulator(theta):
    return theta.to(device)
_,prior = prepare_for_sbi(simulator,priorDist)

/Users/xiaoyuwang/opt/anaconda3/lib/python3.8/site-packages/sbi/utils/user_input_checks_utils.py:390: UserWarning: No prior bounds were passed, consider passing lower_bound
            and / or upper_bound if your prior has bounded support.
  warnings.warn(
/Users/xiaoyuwang/opt/anaconda3/lib/python3.8/site-packages/sbi/utils/user_input_checks_utils.py:69: UserWarning: Prior is lacking mean attribute, estimating prior mean from samples.
  warnings.warn(
/Users/xiaoyuwang/opt/anaconda3/lib/python3.8/site-packages/sbi/utils/user_input_checks_utils.py:80: UserWarning: Prior is lacking variance attribute, estimating prior variance from
                samples...
  warnings.warn(


In [7]:
inference = sbi.inference.SNLE(prior=prior, density_estimator='nsf')
density_estimator = inference.append_simulations(theta, x).train()
posterior = inference.build_posterior(density_estimator)

 Neural network successfully converged after 200 epochs.

In [ ]:
posterior_samples = posterior.sample((10000,),x = x_0,num_chains = 10)

Tuning bracket width...: 100%|██████████| 50/50 [00:25<00:00,  1.94it/s]

Generating samples: 100%|██████████| 10/10 [00:38<00:00,  3.88s/it]

Generating samples:  20%|██        | 202/1000 [14:25<56:23,  4.24s/it]


Generating samples:  42%|████▏     | 422/1000 [30:42<40:12,  4.17s/it]


Generating samples:  64%|██████▍   | 642/1000 [46:27<27:35,  4.62s/it]


Generating samples:  86%|████████▌ | 861/1000 [1:02:59<10:03,  4.34s/it]


Tuning bracket width...: 100%|██████████| 50/50 [00:22<00:00,  2.19it/s]

Generating samples: 100%|██████████| 10/10 [00:48<00:00,  4.85s/it]

Generating samples:   6%|▌         | 59/1000 [05:15<1:26:35,  5.52s/it]


Generating samples:  23%|██▎       | 232/1000 [20:46<1:08:50,  5.38s/it]


In [ ]:
from sbi import analysis as analysis
_ = analysis.pairplot(
    posterior_samples, figsize=(10, 10)
)

In [ ]:
mdic = {"theta_new": posterior_samples.numpy(), "theta_old":theta.numpy(), "sims_old":x.numpy(),"observation":x_0.numpy()}
sio.savemat("SNLE_nsf_syn4_20k.mat", mdic)